# AmBe Experimental Acquisition and Calibration

This notebook describes all of the steps required to take experimental data, parse that data, perform pulse shape discrimination (PSD) cuts, calibrate the data, and generate a pulse height spectrum (PHS) for use in unfolding.  Here calibration data is taken with Na22, Co60, Cs137, and AmBe sources.  The AmBe source also serves as the neutron source to be used for unfolding. 

## Initial Setup of DAQ

Login to bangDAQUser with password !DataUser.

Power on CAEN N8315 Power Supply. 

Open up terminal and launch the DPP_PSD app by typing:
DPP-PSD_ControlSoftware

Connect the DAQ by presing the connect button.  The connection type is USB, and the base address is 32100000.

## Initial Setup of Detector

Turn on the fan (may need to tape to keep it on). Turn on the NIM power supply.

Turn on the Caen HV power supply.  The settings used for this experiment are shown below.  The PMTs use a negative bias with a max bias of 2500V.  The voltage shown below may need to be adjusted based on the calibration to ensure there is no clipping at high incident gamma/neutron energies for other detectors/experiments.  This process is described below.  

<img src="Pics/HVSupply.jpg">

Connect the detector, oscope, and DAQ as shown below. The green BNC connects the detector to the Oscope, the HV connects the detector to the CAEN HV power supply, and the black BNC connects the Oscope to the DAQ.  The termination is 50 $\Omega$. 

<img src="Pics/detSetup.jpg">
<img src="Pics/OscopeSetup.jpg">

Once you a happy with how the detector is performing, remove the connection to the oscope and feed the detector output directly to the DAQ as shown below. NOTE: The choice of A1 or A2 is arbitrary. 

<img src="Pics/detSetup2.jpg">

## DAQ Setup

### Start Acquisition
Sign out the AmBe source and place in front of the detector.

On the DAQ General tab, select Oscilliscope.  Then click on the Oscilliscope Tab and select Plot Display On.

Click on the Channels tab and select Data Acquisition On (located at the bottom of the GUI).

### Set Gain
To determine the neutron (proton) energy to MeVee conversion, use lightTables.getLight(neutron_energy,particle) from the nsd-rootscripts repo.  For this planned 33 MeV on Ta experiment that this calibration is developed for, lightTables.getLight(35,0) = 22.6 MeVee.  

For reference, the AmBe source has neutron up to ~ 10 MeV, which is about 5 MeVee in light space. This can be calculated using the light tables above.

Both the oscope and PSD must be scaled to ensure that the capture the full scale range (FSR) of the intended experiment.  There are two nobs to do this:

1) Bias - The higher the bias the better the PSD <br/>
2) Charge Sensitivity (on the channels tab) - Reduces the resoltuion, but allows for a wider FSR

#### OScope
On the general tab, select Oscilliscope aquisition mode.

On the channels tab, adjust the PSD settings while looking at the oscope traces.  An example settings is shown here:

<img src="Pics/PSDsettings.jpg">

The threshold can be increased until the DAQ does not trigger.  That is the range of the AmBe 10 MeV neutrons. To calculate if the FSR of the experiment is captured:

Max Expected Data Channel = Threshold * Max Proton MeVee from Experiment / Max Proton MeVee from AmBe 

This number needs to be less than the the baseline shown on the scope (~15900 in this case) to be within the FSR of the oscope.  NOTE: The base line can be shifted up to ~16,300 with the DC offset setting.

The settings used are shown above and captured in the DppConfigJamesBevins2017_2_1.txt file.  This can be loaded directly into the software as a starting point.  

#### PSD

A similar excersise must be done with the PSD plot.  Again, the threshold can be increased to see just the tails of the distribution, or it can be determined visually from the PSD plot.

On the general tab, select hisogram acquisition mode. Change to the Histogram tab and turn plotting on.  Select the 2D radio button.

The PSD plot should look something like this:

<img src="Pics/PSD.jpg">

Adjust the PSD gate settings if necessary to improve the PSD.  The settings chose to generate this plot were shown above.

The FSR can be checked by:

Max Expected Data Channel = Threshold * Max Proton MeVee from Experiment / Max Proton MeVee from AmBe 

This must be less than ~35k to be captured in the FSR of the PSD histogram.

### Acquire Data
Click on the Output tab.  Select a file location and name.  Select list, binary format, and energy histogram.  Finally turn dumping on.

Take data until sufficient statistics are build up.  The can be tracked by looking at the 1D energy histogram as shown below.  This is an example of sufficient data needed to perform the calibration.  

<img src="Pics/AmBeResults.jpg">

## Sources

Repeat for each source to be used for calibration. Ensure sufficient statistics are taken for each source.  NOTE: Keep the same settings across all calibraions.

Typical sources used are: AmBe, Co60, CS137, and Na22.  

A background measurement is generally not needed for calibration, but it is a good idea to take one. 

# Calibration Post-Processing Steps:

The rest of this notebook is both descriptive and interactive.  Choose which step to perform in the notebook, or you can run them in root.  NOTE: If using this notebook as a guide to a different data set, check each of variables and commands to make sure they make sense in the context of your data set.


## Convert Binaries to root trees

For each of the files, run the following (the gROOT commands can be copied and run directly in ROOT if preferred):

DPPBinaryParser parser;  
parser.readDatFile("your file name.dat", "output file name.root");

In [1]:
from ROOT import gROOT
import os

path="/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Unfolding/Experiments/AmBe/CalibData/"

In [ ]:
gROOT.ProcessLine('DPPBinaryParser parser;')

for filename in os.listdir(path):
    if filename.endswith(".dat"): 
        name = os.path.splitext(filename)[0]
        gROOT.ProcessLine('parser.readDatFile("{0}{1}.dat", "{0}{1}.root");'.format(path,name))

## Make PSD cuts on AmBe

To perform the calibration, it is necessary to toss out the neutron data from the AmBe data.  Here a simple linear cut is demonstrated.  psdplotfitter.h has some more advanced options if needed.

Each of the gROOT commands can be copied and run directly in ROOT if desired.

First draw the PSD reducing the number of bins to 1024 in both directions:

In [ ]:
gROOT.ProcessLine('TFile f("{0}AmBe_det1.root","update")'.format(path))
gROOT.ProcessLine('eventTree->Draw("(m_amplitude-m_shape)/m_amplitude:m_amplitude>>(4096,0,35000,1024,0,1)","","colz")')

This will open a PSD plot.  

Right click and click SetShowProjectY.  This opens a new canvas.  On the original PSD canvas, you can select different projections which are plotted on the right.  use this to find the location of the minima between the neutron and gamma band.  In this case it is ~0.23.

You can now view those cuts to ensure they are appropriate:

In [ ]:
gROOT.ProcessLine('eventTree->Draw("(m_amplitude-m_shape)/m_amplitude:m_amplitude>>(4096,0,35000,1024,0,1)","(m_amplitude-m_shape)/m_amplitude<0.23","colz")')

Now we can save the cuts.  This generates a gamma and neutron histogram and saves it to the current file.

In [ ]:
gROOT.ProcessLine('eventTree->Draw("m_amplitude>>AmBeGamma(4096,0,35000)","(m_amplitude-m_shape)/m_amplitude<0.23")')
gROOT.ProcessLine('TH1F* AmBeData = (TH1F*) f.Get("AmBeGamma")')

In [ ]:
gROOT.ProcessLine('eventTree->Draw("m_amplitude>>AmBeNeutron(4096,0,35000)","(m_amplitude-m_shape)/m_amplitude>0.23")')
gROOT.ProcessLine('f.Write()')

## Combine the Calibration Data Files

Next, the experimental claibration files need to be combined into a single TFile.  

First pull out each of the pulse height histograms:

In [ ]:
# Cs137
gROOT.ProcessLine('TFile g("{0}Cs137_det1.root","update")'.format(path))
gROOT.ProcessLine('eventTree->Draw("m_amplitude>>Cs137Data(4096,0,35000)")')
gROOT.ProcessLine('g.Write()')
gROOT.ProcessLine('TH1F* Cs137Data = (TH1F*) g.Get("Cs137Data")')

In [ ]:
# Co60
gROOT.ProcessLine('TFile h("{0}Co60_det1.root","update")'.format(path))
gROOT.ProcessLine('eventTree->Draw("m_amplitude>>Co60Data(4096,0,35000)")')
gROOT.ProcessLine('h.Write()')
gROOT.ProcessLine('TH1F* Co60Data = (TH1F*) h.Get("Co60Data")')

In [ ]:
# Na22
gROOT.ProcessLine('TFile i("{0}Na22_det1.root","update")'.format(path))
gROOT.ProcessLine('eventTree->Draw("m_amplitude>>Na22Data(4096,0,35000)")')
gROOT.ProcessLine('i.Write()')
gROOT.ProcessLine('TH1F* Na22Data = (TH1F*) i.Get("Na22Data")')

Now Combine them into a single file:

In [ ]:
gROOT.ProcessLine('TFile j("{0}CalibData.root","new")'.format(path))
gROOT.ProcessLine('Co60Data->Write()')
gROOT.ProcessLine('Cs137Data->Write()')
gROOT.ProcessLine('Na22Data->Write()')
gROOT.ProcessLine('AmBeData->Write()')

## Combine the Calibration Sim Files

Next, the simulated calibration files need to be combined into a single TFile.

## Run the Calibration Software

There are two options for running the calibration sofware:

1) Simulation based calibrations
2) Klein-Nishina based calibration

Each The two options will be explained separately.

### Mode 1 - Simulation Based Calibrations

Required:
1) Vector of pointers to histograms of data
2) Vector of pointers to histograms of simulations of that data in the same order

To run the calibrations, use the startCalibrations script.  NOTE: The repo contains an example that is similar but may differ. This script must be loaded first for both methods.

This script contains user variables that must be modified.  For this mode you can use one of two methods:

1) startGuessSimCalib(): modify the file names
2) startSimCalibration(): modify the file names and each of the parameters

It is recommneded to use startGuessSimCalib(), and that method is explained in this notebook.  However, if you have starting calibration and scale parameters, startSimCalibration() will get you in a closer starting position to begin with. 

First load the script and run the method:

In [2]:
from ROOT import gROOT

gROOT.ProcessLine('.L startCalibration.cpp')
gROOT.ProcessLine('a=startSimCalibration()')

SystemError: long TROOT::ProcessLine(const char* line, int* error = 0) =>
    problem in C++; program state has been reset

### Mode 2 - Klein-Nishina Based Calibrations

Required:
1) Vector of pointers to histograms of data

To run the calibrations, use the startCalibrations script.  NOTE: The repo contains an example that is similar but may differ. This script must be loaded first for both methods.

This script contains user variables that must be modified.  For this mode you can use one of two methods:

1) startGuessKNCalib(): modify the file name and isotopes considered
2) startCalcCalibration(): modify the file names and each of the parameters

It is recommneded to use startGuessKNCalib(), and that method is explained in this notebook.  However, if you have starting calibration and scale parameters, startCalcCalibration() will get you in a closer starting position to begin with. 

First load the script and run the method:

In [3]:
from ROOT import gROOT

gROOT.ProcessLine('.L startCalibration.cpp')
gROOT.ProcessLine('a=startGuessKNCalib()')

SystemError: long TROOT::ProcessLine(const char* line, int* error = 0) =>
    problem in C++; program state has been reset

Set the ranges and the good starting points using the GUI.


In [ ]:
a->findGammaCalibrationConstants()
vector<doubles> thing = a->findGammaCalibrationConstants()


## Apply the Calibration

The x and y are the slope and intercept from the calibration routine.

In [ ]:

HistogramOperations b
vector<TH1*> hosts = b.loadHistograms(“T0Data.root”)
b.applyCalibration(hists[0],x,y)
hists[0]->Draw()
HistogramWriter writer;
writer.()

In [ ]:

eventTree->Draw("m_shape:m_amplitude>>(1024,0,70000,1024,0,35000)","","colz")

#draw psd
eventTree->Draw("(m_amplitude-m_shape)/m_amplitude:m_amplitude>>(1024,0,35000,1024,0,1)","","colz")

#view psd linear cuts
eventTree->Draw("(m_amplitude-m_shape)/m_amplitude:m_amplitude>>(1024,0,35000,1024,0,1)","(m_amplitude-m_shape)/m_amplitude>0.16","colz")

#generate phs with linear psd cuts
eventTree->Draw("m_amplitude>>phs","(m_amplitude-m_shape)/m_amplitude>0.16")